In [5]:
import numpy as np
import time
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from heapq import *
import networkx as nx
from collections import defaultdict

# Question 1

In [2]:
##create adj matrix for p-ER graph with K nodes and probabilty p
def p_ER(k, p):
    mat = np.zeros((k,k))
        
    for i in range(k):
        for j in range(i+1, k):
            add = np.random.binomial(1, p, 1)[0]
            if(add == 1):
                mat[i, j] = 1
                mat[j, i] = 1
    return mat

# Question 2

In [86]:
#compute connectivity using irreducibility
#we need to check that from the adj matrix A is true that
#I + A + A^2 + ... + A^n-1 >0

def is_irreducible(A):
    n = len(A)
    I = np.identity(n)
    Sum = I #start the sum with I
    A_new=A.copy()
    
    for i in range(1,n): 
        Sum += A_new #we ad A to the sum...
        A_new = np.inner(A_new, A) #...and update it
        
    if( np.min(Sum) > 0 ): # check if there are some 0s in the Sum matrix
        return ('Graph connected')
    else:
        return ('Graph not connected')

In [104]:
#compute connectivity using laplacian method
#we pass the graph in input as a adj matrix A

def L_method(A):
    #we need to compute the laplacian matrix as L=D-A
    #where D is a diagonal matrix having the sum of rows of A
    D = np.diag(sum(A)) 
    L = D - A
    
    print(np.sort(np.linalg.eigvals(L)))
    ###WARNING!! IDK WHY IT WORKS ONLY USING SECOND SMALLEST EIGVAL
    if( np.sort(np.linalg.eigvals(L))[0] > 0): #check if eigenvalues are non negative
        return ('Graph connected')
    else:
        return ('Graph not connected')